In [0]:
!pip install tensorflow==1.14.0

     |█████████████████████████████▊  | 101.3MB 1.9MB/s eta 0:00:05

In [0]:


from matplotlib import pyplot as plt
import cv2
import os
import numpy as np
import torch

from keras.utils import np_utils
from keras.callbacks import History 

from datetime import datetime

import tensorflow as tf


import os
from google.colab import drive

Using TensorFlow backend.
/usr/local/lib/python3.6/dist-packages/tensorflow/python/framework/dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/usr/local/lib/python3.6/dist-packages/tensorflow/python/framework/dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/usr/local/lib/python3.6/dist-packages/tensorflow/python/framework/dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/usr/local/lib/python3.6/dist-packages/tensorflow/python/framework/dtypes.py:519: FutureWarn

In [0]:

drive.mount('/content/drive')

COMP_PATH = "/content/drive/My Drive/Colab Notebooks/NUS/CS5242 Neural Networks and Deep Learning/Project"
# COMP_PATH = 'C:/Users/moong/Desktop/CS5242 Project/'
# COMP_PATH = 'C:\\Users\\Calvin\\Documents\\Python Scripts\\NUS\\CS5242 Neural Networks and Deep Learning\\Project'

os.chdir(COMP_PATH)
os.listdir()

tf.__version__

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


'1.14.0'

In [0]:
def add_video_segment(video_segment, segment_label, all_video_segments, all_video_labels, target_len):
    video_len = len(video_segment)
    if (video_len > 0) and (segment_label != 0):
        if (video_len < target_len):
            padded_video_segment = np.zeros((target_len, 400))
            padded_video_segment[0:len(video_segment)] = video_segment
        elif (video_len > target_len):
            # Get the middle part
            start_idx = (video_len-target_len)//2
            padded_video_segment = video_segment[start_idx:start_idx+target_len]
        else:
            padded_video_segment = video_segment
        all_video_segments.append(padded_video_segment)
        all_video_labels.append(segment_label)
    return all_video_segments, all_video_labels

def get_video_segment_by_len(video_group_list, label_group_list, target_len):
    all_video_segments = []
    all_video_labels = []
    for i, label_group in enumerate(label_group_list):
        video_group = video_group_list[i]
        video_segment = []
        segment_label = label_group_list[0]
        
        idx = 0
        for j, label in enumerate(label_group):
            
            if (label != segment_label):
                all_video_segments, all_video_labels = add_video_segment(
                    video_segment, segment_label, all_video_segments, all_video_labels, target_len)
                segment_label = label
                video_segment = []
                idx = 0
            video_segment.append(video_group[j])
            idx += 1
            if idx == target_len:
                all_video_segments, all_video_labels = add_video_segment(
                    video_segment, segment_label, all_video_segments, all_video_labels, target_len)
                video_segment = []
                idx = 0

        all_video_segments, all_video_labels = add_video_segment(
            video_segment, segment_label, all_video_segments, all_video_labels, target_len)
    
    return np.array(all_video_segments), np.array(all_video_labels)
    
def load_data(split_load, actions_dict, GT_folder, DATA_folder, datatype = 'training', target_len = 100):
    file_ptr = open(split_load, 'r')
    content_all = file_ptr.read().split('\n')[1:-1]
    content_all = [x.strip('./data/groundTruth/') + 't' for x in content_all]
    all_tasks = ['tea', 'cereals', 'coffee', 'friedegg', 'juice', 'milk', 'sandwich', 'scrambledegg', 'pancake', 'salat']
    
    if datatype == 'training':
        data_breakfast = []
        labels_breakfast = []
        for content in content_all:
        
            file_ptr = open( GT_folder + content, 'r')
            curr_gt = file_ptr.read().split('\n')[:-1]
            # label_seq, length_seq = get_label_length_seq(curr_gt)

            loc_curr_data = DATA_folder + os.path.splitext(content)[0] + '.gz'
        
            curr_data = np.loadtxt(loc_curr_data, dtype='float32')
            label_curr_video = []
            for iik in range(len(curr_gt)):
                label_curr_video.append( actions_dict[curr_gt[iik]] )
         
            data_breakfast.append(curr_data)
            labels_breakfast.append(label_curr_video)
    
        # labels_uniq, labels_uniq_loc = get_label_bounds(labels_breakfast)

        all_video_segments, all_video_labels = get_video_segment_by_len(data_breakfast, labels_breakfast, target_len)

        print("Finish Load the Training data and labels!!!")     
        return all_video_segments, all_video_labels

        # return  data_breakfast, labels_uniq

        # return  data_breakfast, labels_breakfast
    if datatype == 'test':
        data_breakfast = []
        for content in content_all:
        
            loc_curr_data = DATA_folder + os.path.splitext(content)[0] + '.gz'
        
            curr_data = np.loadtxt(loc_curr_data, dtype='float32')
            
            data_breakfast.append(torch.tensor(curr_data,  dtype=torch.float64 ) )
    
        print("Finish Load the Test data!!!")
        return data_breakfast


def find_end_idx(y_train, start_idx, target_len):
    orig_start_idx = start_idx
    curr_label = y_train[start_idx]
    # print(curr_label)
    counter = 0
    # print("start_idx:", start_idx)
    # print("adfdasfdd:", target_len)
    while counter < target_len and \
          start_idx < len(y_train) and \
          curr_label == y_train[start_idx]:
        # print(start_idx)
        # print(y_train[start_idx])
        start_idx += 1
        counter += 1

    if counter >= target_len:
        return orig_start_idx + target_len
    elif start_idx >= len(y_train):
        return len(y_train)
    else:
        return start_idx

def pad_x(x, target_len):
    padded_x = np.zeros((target_len, 1, 400))
    padded_x[0:len(x)] = x
    return padded_x

def pad_y(y, target_len):
    padded_y = np.zeros((target_len, 1))
    padded_y[0:len(y)] = y
    return padded_y

In [0]:
from read_datasetBreakfast import read_mapping_dict

COMP_PATH = "/content/drive/My Drive/Colab Notebooks/NUS/CS5242 Neural Networks and Deep Learning/Project"

''' 
training to load train set
test to load test set
'''
split = 'training'
#split = 'test'
train_split =  os.path.join(COMP_PATH, 'splits/5 train.split1.bundle') #Train Split
# valid_split =  os.path.join(COMP_PATH, 'splits/2 valid.split1.bundle') #Train Split 
# train_split =  os.path.join(COMP_PATH, 'splits/300 train.split1 - P16_48.bundle') #Train Split
# valid_split =  os.path.join(COMP_PATH, 'splits/90 valid.split1 - P49_54.bundle') # Valid Split 
# train_split =  os.path.join(COMP_PATH, 'splits/900 train.split1 - P16_48.bundle') #Train Split
# valid_split =  os.path.join(COMP_PATH, 'splits/150 valid.split1 - P49_54.bundle') # Valid Split 
# train_split =  os.path.join(COMP_PATH, 'splits/train.split1 - Orig.bundle') #Train Split
test_split  =  os.path.join(COMP_PATH, 'splits/test.split1.bundle') #Test Split
GT_folder   =  os.path.join(COMP_PATH, 'groundTruth/') #Ground Truth Labels for each training video 
DATA_folder =  os.path.join(COMP_PATH, 'data/') #Frame I3D features for all videos
mapping_loc =  os.path.join(COMP_PATH, 'splits/mapping_bf.txt') 

segment_len = 1

actions_dict = read_mapping_dict(mapping_loc)
if  split == 'training':
    startTime = datetime.now()
    X_data, Y_data = load_data( train_split, actions_dict, GT_folder, DATA_folder, datatype = split, target_len = segment_len) #Get features and labels
    # labels_uniq, labels_uniq_loc = get_label_bounds(data_labels)
    endTime = datetime.now()
    print("******** Total Time to load file:",(endTime - startTime))
if  split == 'test':
    data_feat = load_data( test_split, actions_dict, GT_folder, DATA_folder, datatype = split) #Get features only


Y_data = np.reshape(Y_data, (Y_data.shape[0], 1))

# print(len(data_feat))
# print(len(data_labels))
# print(data_feat)
# print(data_labels)

Finish Load the Training data and labels!!!
******** Total Time to load file: 0:00:01.929998


In [0]:
# print(Y.shape)

# label_dict = {}
# label_len = {}

# target_len_tmp = 99999999
# start_idx = 0
# max_len = 0
# end_idx = find_end_idx(Y, start_idx, target_len_tmp)
# while start_idx < len(Y):
#     this_len = end_idx - start_idx
#     if (this_len > max_len):
#         max_len = this_len
#         print("new max len:", max_len)
#     start_idx = end_idx
#     if start_idx < len(Y):
#         end_idx = find_end_idx(Y, start_idx, target_len_tmp)
    
#         curr_label = int(Y[start_idx])
#         if curr_label in label_dict:
#             label_dict[curr_label] += 1
#             if this_len > label_len[curr_label]:
#                 label_len[curr_label] = this_len
#         else:
#             label_dict[curr_label] = 1
#             label_len[curr_label] = this_len
    

# print(max_len)
# total_occurence = 0
# for i, curr_label in enumerate(label_dict):
#     print("curr_label: {}; occurence: {}; max length: {}".format(curr_label, label_dict[curr_label], label_len[curr_label]))
#     total_occurence += label_dict[curr_label]
    
# print("total_occurence:", total_occurence)



In [0]:
# Y = np_utils.to_categorical(Y, num_classes=48)
# # print(Y.shape)
# Y = np.reshape(Y, (Y.shape[0], 48))

# print(X.shape)
# print(Y.shape)



In [0]:
target_len = 3000

# n_hidden_nodes = target_len//2
n_hidden_nodes = 100
n_dense_nodes = 50
n_features = 400
n_timesteps = target_len
n_outputs = 48

n_epoch = 1
batch_size = 128
init_lr = 0.01

clipnorm=1.0
clipvalue=0.5

input_dropout = 0.4
recurrent_dropout = 0.4


total_num_of_segments = 7068
num_display = 50
num_of_itr = 100

In [0]:
from keras import optimizers
from keras import regularizers

# from keras.models import Model, Sequential
# from keras.layers import Dense, Flatten, Dropout, LSTM
from keras.models import Sequential
from keras.layers import Dense, LSTM, Bidirectional, BatchNormalization, TimeDistributed, Flatten



lr = tf.compat.v1.train.exponential_decay(
    init_lr,                # Base learning rate.
    batch_size,  # Current index into the dataset.
    8000,          # Decay step.
    0.8,                # Decay rate.
    staircase=True)

adam = optimizers.Adam(lr=lr, clipnorm=clipnorm, clipvalue=clipvalue)



model = Sequential()
# model.add(Bidirectional(LSTM(n_hidden_nodes, 
#                              input_shape=(n_timesteps, n_features), 
#                              kernel_initializer='random_uniform',
#                              recurrent_initializer='random_uniform',
#                             #  kernel_regularizer=regularizers.l2(0.01), 
#                             #  recurrent_regularizer=regularizers.l2(0.01) 
#                              )))
# model.add(Bidirectional(LSTM(n_hidden_nodes, return_sequences=True, input_shape=(n_timesteps, n_features))))
model.add(LSTM(n_hidden_nodes, return_sequences=True, input_shape=(n_timesteps, n_features)))
# model.add(Bidirectional(LSTM(n_hidden_nodes, return_sequences=True, input_shape=(n_timesteps, n_features), dropout=input_dropout, recurrent_dropout=recurrent_dropout)))
model.add(BatchNormalization())
# model.add(Dropout(0.2))
# model.add(LSTM(20))
# model.add(BatchNormalization())
# model.add(Dropout(0.2))
model.add(TimeDistributed(Dense(n_dense_nodes, activation='relu')))
# model.add(Dropout(0.2))
# model.add(Dense(200, activation='relu'))
model.add(TimeDistributed(Dense(n_outputs, activation='softmax')))

model.compile(loss='sparse_categorical_crossentropy', metrics=['categorical_accuracy'], optimizer=adam)


# Split train vs valid data set
ratio = 0.9
x_train_size = int(X.shape[0]*ratio)
x_train = X[:x_train_size]
y_train = Y[:x_train_size]
x_valid = X[x_train_size:Y.shape[0]]
y_valid = Y[x_train_size:Y.shape[0]]

print("x_train size: ", x_train.shape[0])
print("x_valid size: ", x_valid.shape[0])


history = History()

x_train_size = x_train.shape[0]
# start_idx = 0
# end_idx = start_idx + target_len
total_loss = 0
total_acc = 0
counter = 0


for i in range(n_epoch):
    total_loss = 0
    total_acc = 0
    counter = 0

    start_idx = 0
    end_idx = find_end_idx(y_train, start_idx, target_len)
    startTime = datetime.now()
    epoch_itr = 1
    while True:
        
        x_train_one_step = x_train[start_idx:end_idx]
        y_train_one_step = y_train[start_idx:end_idx]

        # print(y_train_one_step.shape)
        curr_label = y_train_one_step[0]

        x_train_one_step = pad_x(x_train_one_step, target_len)
        y_train_one_step = pad_y(y_train_one_step, target_len)

        # y_train_one_step = np_utils.to_categorical(y_train_one_step, num_classes=n_outputs)

        x_train_one_step = np.reshape(x_train_one_step, (1, x_train_one_step.shape[0], x_train_one_step.shape[2]))
        y_train_one_step = np.reshape(y_train_one_step, (1, y_train_one_step.shape[0], y_train_one_step.shape[1]))

        history = model.fit(x_train_one_step, y_train_one_step, epochs=1, batch_size=batch_size, verbose=0)

        # print(history.history.keys())
        # break
        train_loss_list = history.history['loss']
        train_acc_list  = history.history['categorical_accuracy']

        for i, train_loss in enumerate(train_loss_list):
            total_loss += train_loss
            total_acc += train_acc_list[i]
            counter += 1
            average_loss = total_loss / counter
            average_acc = total_acc / counter

        if (epoch_itr % num_of_itr == 0):
            local_loss = total_loss/counter
            local_acc = total_acc/counter
            # print('***** Iteration {}, ===== acc {:.2f}, loss {:.2f}\n'.format(epoch_itr, local_acc, local_loss))

            for i, train_loss in enumerate(train_loss_list):
                print('***** Label {}, ===== start_idx {}, end_idx {}'.format(curr_label, start_idx, end_idx))
                print('***** Iteration {}, ===== acc {:.3f}, loss {:.3f}'.format(epoch_itr, train_acc_list[i], train_loss))
                print('***** Running Average ===== acc {:.3f}, loss {:.3f}\n'.format(average_acc, average_loss))

        epoch_itr += 1
        if end_idx >= x_train_size:
            break
        else:
            start_idx = end_idx
            end_idx = find_end_idx(y_train, start_idx, target_len)
 
        # print(start_idx)
        # print(end_idx)

    endTime = datetime.now()
    # print("*** Time took: "+str(endTime-startTime))

    average_loss = total_loss / counter
    average_acc = total_acc / counter
    print('***** Epoch {}, Time took: {}, ===== acc {:.3f}, loss {:.3f}\n'.format(i, str(endTime-startTime), average_acc, average_loss))
    # print('***** Epoch %, Time took: %, ===== acc %.2f, loss %.2f\n' % (i, str(endTime-startTime), average_acc, average_loss), end='')


    # end_idx = start_idx + target_len

    print('\n# Evaluate on validation data')
    x_valid_size = x_valid.shape[0]
    start_idx = 0
    end_idx = find_end_idx(y_valid, start_idx, target_len)
    total_valid_loss = 0
    total_valid_acc = 0
    counter = 0
    while True:
        x_valid_one_step = x_valid[start_idx:end_idx]
        y_valid_one_step = y_valid[start_idx:end_idx]

        x_valid_one_step = pad_x(x_valid_one_step, target_len)
        y_valid_one_step = pad_y(y_valid_one_step, target_len)

        # y_valid_one_step = np_utils.to_categorical(y_valid_one_step, num_classes=n_outputs)

        x_valid_one_step = np.reshape(x_valid_one_step, (1, x_valid_one_step.shape[0], x_valid_one_step.shape[2]))
        y_valid_one_step = np.reshape(y_valid_one_step, (1, y_valid_one_step.shape[0], y_valid_one_step.shape[1]))

        results = model.evaluate(x_valid_one_step, y_valid_one_step, verbose=2)

        # print(type(results))
        # print(results)
        total_valid_loss += results[0]
        total_valid_acc += results[1]
        counter += 1

        # print('Validation loss, Validation acc:', results)

        if end_idx >= x_valid_size:
            break
        else:
            start_idx = end_idx
            end_idx = find_end_idx(y_valid, start_idx, target_len)

    average_valid_loss = total_valid_loss/counter
    average_valid_acc = total_valid_acc/counter

    print('***** Running Average ===== acc {:.3f}, loss {:.3f}\n'.format(average_valid_acc, average_valid_loss))
    # print('Validation loss, Validation acc:', results)


    # print(history)
#     print('acc %.2f, loss %.2f' % (acc, loss), end='')

    
# print('\n# Evaluate on test data')
# x_valid_size = x_valid.shape[0]
# start_idx = 0
# end_idx = start_idx + target_len
# while end_idx <= x_valid_size:
#     x_valid_one_step = x_valid[start_idx:end_idx]
#     y_valid_one_step = y_valid[start_idx:end_idx]
#     x_valid_one_step = np.reshape(x_valid_one_step, (1, x_valid_one_step.shape[0], x_valid_one_step.shape[2]))
#     y_valid_one_step = np.reshape(y_valid_one_step, (1, y_valid_one_step.shape[0], y_valid_one_step.shape[1]))

#     results = model.evaluate(x_valid_one_step, y_valid_one_step, verbose=2)

#     start_idx = end_idx
#     end_idx = start_idx + target_len

# print('test loss, test acc:', results)

model.summary()

model.save('LSTM_Main_3000.h5')


x_train size:  6559
x_valid size:  729
***** Epoch 0, Time took: 0:00:17.661282, ===== acc 0.694, loss 3.042


# Evaluate on validation data
***** Running Average ===== acc 0.879, loss 0.722

Model: "sequential_6"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm_6 (LSTM)                (None, 3000, 100)         200400    
_________________________________________________________________
batch_normalization_6 (Batch (None, 3000, 100)         400       
_________________________________________________________________
time_distributed_11 (TimeDis (None, 3000, 50)          5050      
_________________________________________________________________
time_distributed_12 (TimeDis (None, 3000, 48)          2448      
Total params: 208,298
Trainable params: 208,098
Non-trainable params: 200
_________________________________________________________________


In [0]:
# from keras import optimizers
# from keras import regularizers

# # from keras.models import Model, Sequential
# # from keras.layers import Dense, Flatten, Dropout, LSTM
# from keras.models import Sequential
# from keras.layers import Dense, LSTM, Bidirectional, BatchNormalization, TimeDistributed, Flatten



# lr = tf.compat.v1.train.exponential_decay(
#     init_lr,                # Base learning rate.
#     batch_size,  # Current index into the dataset.
#     8000,          # Decay step.
#     0.8,                # Decay rate.
#     staircase=True)

# adam = optimizers.Adam(lr=lr, clipnorm=clipnorm, clipvalue=clipvalue)



# model = Sequential()
# # model.add(Bidirectional(LSTM(n_hidden_nodes, 
# #                              input_shape=(n_timesteps, n_features), 
# #                              kernel_initializer='random_uniform',
# #                              recurrent_initializer='random_uniform',
# #                             #  kernel_regularizer=regularizers.l2(0.01), 
# #                             #  recurrent_regularizer=regularizers.l2(0.01) 
# #                              )))
# # model.add(Bidirectional(LSTM(n_hidden_nodes, return_sequences=True, input_shape=(n_timesteps, n_features))))
# model.add(LSTM(n_hidden_nodes, return_sequences=True, input_shape=(n_timesteps, n_features)))
# # model.add(Bidirectional(LSTM(n_hidden_nodes, return_sequences=True, input_shape=(n_timesteps, n_features), dropout=input_dropout, recurrent_dropout=recurrent_dropout)))
# model.add(BatchNormalization())
# # model.add(Dropout(0.2))
# # model.add(LSTM(20))
# # model.add(BatchNormalization())
# # model.add(Dropout(0.2))
# model.add(TimeDistributed(Dense(n_dense_nodes, activation='relu')))
# # model.add(Dropout(0.2))
# # model.add(Dense(200, activation='relu'))
# model.add(TimeDistributed(Dense(n_outputs, activation='softmax')))

# model.compile(loss='sparse_categorical_crossentropy', metrics=['categorical_accuracy'], optimizer=adam)


# # Split train vs valid data set
# ratio = 0.9
# x_train_size = int(X_data.shape[0]*ratio)
# x_train = X_data[:x_train_size]
# y_train = Y_data[:x_train_size]
# x_valid = X_data[x_train_size:Y_data.shape[0]]
# y_valid = Y_data[x_train_size:Y_data.shape[0]]

# print("x_train size: ", x_train.shape[0])
# print("x_valid size: ", x_valid.shape[0])


# history = History()


# history = model.fit(X_data, Y_data, epochs=5, batch_size=batch_size, validation_split=0.2, verbose=2)





# model.summary()

# model.save('LSTM_Main_3000.h5')


x_train size:  6559
x_valid size:  729
Train on 5830 samples, validate on 1458 samples
Epoch 1/5
 - 3s - loss: 0.6179 - categorical_accuracy: 0.0000e+00 - val_loss: 14.7286 - val_categorical_accuracy: 0.0000e+00
Epoch 2/5
 - 1s - loss: 0.2318 - categorical_accuracy: 0.0000e+00 - val_loss: 15.0638 - val_categorical_accuracy: 0.0000e+00
Epoch 3/5
 - 1s - loss: 0.2327 - categorical_accuracy: 0.0000e+00 - val_loss: 14.7706 - val_categorical_accuracy: 0.0000e+00
Epoch 4/5
 - 1s - loss: 0.1876 - categorical_accuracy: 0.0000e+00 - val_loss: 14.7609 - val_categorical_accuracy: 0.0000e+00
Epoch 5/5
 - 1s - loss: 0.2042 - categorical_accuracy: 0.0000e+00 - val_loss: 14.9838 - val_categorical_accuracy: 0.0000e+00
Model: "sequential_8"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm_8 (LSTM)                (None, 1, 100)            200400    
_________________________________________________________________
ba

In [0]:
# target_len = 1

# # n_hidden_nodes = target_len//2
# n_hidden_nodes = 100
# n_dense_nodes = 50
# n_features = 400
# n_timesteps = target_len
# n_outputs = 48

# n_epoch = 1
# batch_size = 128
# init_lr = 0.01

# clipnorm=1.0
# clipvalue=0.5

# input_dropout = 0.4
# recurrent_dropout = 0.4


# total_num_of_segments = 7068
# num_display = 50
# num_of_itr = 100

# Y_data = np.reshape(Y_data, (Y_data.shape[0],1,1))
# print(Y_data.shape)

(7288, 1, 1)


In [0]:
# a = []
# while True:
#     a.append("1")